# Below is the code with an explanation of each code section

All necessary imports

Required libraries:
- pandas
- pickle
- holidays
- sklearn

In [308]:
import pandas as pd
import pickle
import holidays

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

- Read the file
- Create a `DateTime` column by combining the `Date` and `Hour` columns
- Set the newly created column as index

In [309]:
demand = pd.read_csv("Sample Dataset.csv")
demand["DateTime"] = pd.to_datetime(demand.Date.astype(str) + " " + demand.Hour.astype(str) + ":00:00")
demand.set_index("DateTime", drop=True, inplace=True)
demand

,Date,Weekday,Hour,HOEP,Ontario_Demand,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station
DateTime,,,,,,,,,,,,
2016-01-01 00:00:00,2016-01-01,Friday,0,0.49,14023,-0.3,-5.62,20,-3.18,70,-5.1,99.67
2016-01-01 01:00:00,2016-01-01,Friday,1,-1.09,13417,-0.3,-6.30,25,-3.48,68,-5.5,99.63
2016-01-01 02:00:00,2016-01-01,Friday,2,-2.41,12968,-0.4,-6.55,26,-3.43,73,-4.7,99.59
2016-01-01 12:00:00,2016-01-01,Friday,12,0.00,14215,-2.1,-8.72,23,-4.86,76,-5.8,99.39
2016-01-01 13:00:00,2016-01-01,Friday,13,0.00,14443,-2.4,-8.23,19,-5.09,84,-4.7,99.35
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-31 19:00:00,2020-12-31,Thursday,19,36.85,17384,0.2,-4.02,13,-2.69,74,-3.8,100.66
2020-12-31 20:00:00,2020-12-31,Thursday,20,19.69,16783,-0.2,-5.59,16,-3.58,65,-6.0,100.76
2020-12-31 21:00:00,2020-12-31,Thursday,21,20.78,16154,-3.0,-7.44,12,-3.67,80,-6.0,100.81


Create a `Is_Weekday` and `Is_Holiday` column since there is a difference is electricity use for those days.

In [310]:
weekend_days = {"Saturday", "Sunday"}
demand["Is_Weekday"] = demand.Weekday.apply(lambda x: 0 if x in weekend_days else 1)
ontario_holidays = holidays.country_holidays("CA", subdiv="ON")
demand["Is_Holiday"] = demand.Date.apply(lambda x: 0 if ontario_holidays.get(x) == None else 1)

Filter the data on July and August only since they are the target of the prediction.

In [311]:
summer_demand = demand[demand.index.month.isin([7, 8])]
summer_demand

,Date,Weekday,Hour,HOEP,Ontario_Demand,Temperature,Windchill_Index,Wind_Speed,Humidex,Relative_Humidity,Dew_Point,Pressure_Station,Is_Weekday,Is_Holiday
DateTime,,,,,,,,,,,,,,
2016-07-01 00:00:00,2016-07-01,Friday,0,15.40,14919,20.5,20.73,13,21.21,36,5.1,99.32,1,1
2016-07-01 01:00:00,2016-07-01,Friday,1,19.00,12320,19.6,20.04,7,20.44,40,5.8,99.31,1,1
2016-07-01 02:00:00,2016-07-01,Friday,2,0.79,12836,18.8,18.79,10,19.37,42,5.8,99.25,1,1
2016-07-01 03:00:00,2016-07-01,Friday,3,10.68,12502,17.6,18.26,5,18.79,46,6.0,99.21,1,1
2016-07-01 04:00:00,2016-07-01,Friday,4,3.61,12134,17.2,18.09,2,18.49,48,6.2,99.19,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-31 19:00:00,2020-08-31,Monday,19,21.71,17960,21.5,21.89,20,26.94,66,15.1,99.22,1,0
2020-08-31 20:00:00,2020-08-31,Monday,20,21.16,17561,20.4,20.54,19,26.63,75,15.8,99.15,1,0
2020-08-31 21:00:00,2020-08-31,Monday,21,18.90,17331,20.4,20.26,17,25.40,78,16.6,99.16,1,0


Create the training dataset with all features

In [312]:
features = ["Is_Weekday", "Is_Holiday", "Hour", "HOEP", "Temperature", "Windchill_Index", "Wind_Speed", "Humidex", "Relative_Humidity", "Dew_Point", "Pressure_Station"]
x_train = summer_demand[features]
y_train = summer_demand.Ontario_Demand

We need to predict a numerical value so this is a linear regression problem.

The problem description mentions that "_The acceptable error rate must be less than 5% (AE < 500 MW, or accuracy more than 95%)_". 

In my opinion, the metric coresponding to that is the Mean Absolute Percentage Error.

I selected the LinearRegression model for a first cross evaluation and it satisfies the requirements.

In [313]:
evaluation_model = LinearRegression()
scores = cross_val_score(evaluation_model, x_train, y_train, scoring='neg_mean_absolute_percentage_error', cv=5)
print("MAPE=", sum(scores)/len(scores))

MAPE= -0.04754631030945496


Train the model and all data and save it as a pickle file for future use.

In [314]:
full_model = LinearRegression()
full_model.fit(x_train, y_train)

pickle.dump(full_model, open("trained_model.pickle", "ab"))